# Génération automatique de la base de données à partir de sauvegardes CSV
### Margaux BRULIARD
### 18 mars 2021

L'ensemble des fichiers sont contenus dans le dossier `save_database`

In [1]:
import csv
import os
import sqlite3 as bdd

On commence par re-créer la base de données `database.db`

In [6]:
path_db = '../database_2.db'

# on vérifie si la base de données existe déjà
if os.path.exists(path_db):
    x = input("La BDD existe déjà. Etes vous sur de vouloir écraser les données ? [Y/n]")
    if (x == 'Y'):
        os.system('rm ../database_2.db')
    else:
        exit()
# ....

os.system("touch ../database_2.db") # creation du fichier
connexion = bdd.connect("../database_2.db") # connexion à la base de données
curseur = connexion.cursor()

La BDD existe déjà. Etes vous sur de vouloir écraser les données ? [Y/n]n


On lit le fichier `creation.sql` pour lancer les requêtes dans la base de données

In [3]:
queries = None
with open('save_database/creation.sql', 'r') as fichier:
    queries = fichier.read()
# ...
curseur.executescript(queries)
connexion.commit()

Maintenant que les tables sont construites, nous allons pouvoir les remplir

In [4]:
table_names = ['categories', 'individus', 'films', 'distribution', 'notes', 'forum']
for table in table_names:
    with open('save_database/save_' + table + '.csv', newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        datas = list(csv_reader)
        
        # on extrait la 1ere ligne qui correspond aux noms des colonnes
        columns = datas[0]
        datas = datas[1:]
        
        # on traite les donnees pour que chaque vide soit considéré comme un 'NULL' dans la bdd
        for line in datas:
            for i in range(0, len(line)):
                if line[i] == '':
                    line[i] = 'NULL'
                # ...
            # ...
        # ...
        
        # maintenant on peut faire des queries
        for line in datas:
            query = "INSERT INTO {} {} VALUES {}".format(table, tuple(columns), tuple(line))
            curseur.execute(query)
        # ...
        connexion.commit()    
    # ...
# ...


Fermeture de base de données

In [5]:
connexion.close()